In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from sklearn.ensemble import RandomForestRegressor
from dash import dcc
from dash import html
import numpy as np
import pandas as pd
import joblib
from dash.dependencies import Input, Output

C:\Users\jeroc\AppData\Local\Temp\ipykernel_36256\2562867785.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\jeroc\AppData\Local\Temp\ipykernel_36256\2562867785.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [26]:
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [3]:
model = joblib.load('model.joblib')
scaler = joblib.load('scaler.joblib')

In [27]:
# Create the app
app = dash.Dash()

In [28]:
# Create the layout
app.layout = html.Div([
    html.H1('Walmart Sales Prediction'),
    html.Div([
        html.H3('Store:'),
        dcc.Input(id='Store', type='number', value=1, min=1, max=2)
    ]),
    html.Div([
        html.H3('Dept:'),
        dcc.Input(id='Dept', placeholder='', type='number', value=1, min=1, max=3)
    ]),
    html.Div([
        html.H3('Temperature:'),
        dcc.Input(id='Temperature', placeholder='', type='number', value=10, min=1, max=10)
    ]),
    html.Div([
        html.H3('Fuel_Price:'),
        dcc.Input(id='Fuel_Price', placeholder='', type='number', value=2, min=1, max=5)
    ]),
    html.Div([
        html.H3('MarkDown2:'),
        dcc.Input(id='MarkDown2', placeholder='', type='number', value=100, min=5, max=10)
    ]),
    html.Div([
        html.H3('MarkDown3:'),
        dcc.Input(id='Markdown3', placeholder='', type='number', value=100, min=5, max=10)
    ]),
    html.Div([
        html.H3('Unemployment:'),
        dcc.Input(id='Unemployment', placeholder='', type='number', value=5, min=5, max=10)
    ]),

    html.Div([
        html.Button('Predicted Sales:'),
        #dcc.output(id='Predicted_Sales', type='text')
        html.Div(id='predict-button')
    ])
])

In [29]:
def get_prediction(Store, Dept, Temperature, Fuel_Price, MarkDown2, MarkDown3, MarkDown4, Unemployment):
    '''takes the inputs from the user and produces the sales prediction'''

    cols = ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'Unemployment']

    ddf = pd.DataFrame(data = np.zeros((1,len(cols))), columns = cols)

    ## get the numeric characteristics
    ddf.loc[0,'Store'] = Store
    ddf.loc[0,'Dept'] = Dept
    ddf.loc[0,'Temperature'] = Temperature
    ddf.loc[0,'Fuel_Price'] = Fuel_Price
    ddf.loc[0,'MarkDown2'] = MarkDown2
    ddf.loc[0,'MarkDown3'] =MarkDown3
    ddf.loc[0,'MarkDown4'] = MarkDown4
    ddf.loc[0,'Unemployment'] = Unemployment

    ## Scale the numerical features using the trained scaler
    numerical_features = ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'Unemployment']
    ddf.loc[:,numerical_features] = scaler.transform(ddf.loc[:,numerical_features])

    ## Get the predictions using our trained neural network
    prediction = model.predict(ddf.values).flatten()[0]

    ## Transform the log-sales to sales
    prediction = np.exp(prediction)

    return prediction

In [30]:
# Create the callback
@app.callback(
    Output('Predicted_Sales', 'children'),
    [Input('Store', 'value'),
     Input('Dept', 'value'),
     Input('Temperature', 'value'),
     Input('Fuel_Price', 'value'),
     Input('MarkDown2', 'value'),
     Input('Markdown3', 'value'),
     Input('Unemployment', 'value')])
def predict_sales(store, dept, temperature, fuel_price, markdown2, markdown3, unemployment):
    # Create the input data
    features = {
        'Store': store,
        'Dept': dept,
        'Temperature': temperature,
        'Fuel_Price': fuel_price,
        'MarkDown2': markdown2,
        'Markdown3': markdown3,
        'Unemployment': unemployment
    }

    # Make the prediction
def show_prediction(Store, Dept, Temperature, Fuel_Price, MarkDown2, MarkDown3, MarkDown4, Unemployment):
    pred = get_prediction(Store, Dept, Temperature, Fuel_Price, MarkDown2, MarkDown3, MarkDown4, Unemployment)
    return str("Predicted Price: {:,}".format(pred))

# Run the app
app.run_server(debug=False)